<a href="https://colab.research.google.com/github/Nadeem-Anwar/Nadeem-Anwar/blob/main/Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#To mount google drive to google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import os
import cv2
from matplotlib import pyplot
source = '/content/drive/MyDrive/BreaKHis_v1/histology_slides/breast'
destination = '/content/sample_data'

try:
  benign = os.mkdir(destination + '/benign')
  malign = os.mkdir(destination + '/malignant')
except FileExistsError:
  print('Destination folders already exist')

Destination folders already exist


In [33]:
os.chdir(source)
for d in os.listdir():
  if d == 'benign':
    os.chdir(d)
    dest = destination + '/benign/'
  elif d == 'malignant':
    os.chdir(d)
    dest = destination + '/malignant/'
  else:
    continue
  #print(os.getcwd(),dest, 'in bm')
  for dd in os.listdir():
    if dd == 'SOB':
      os.chdir(dd)
      os.getcwd()
    else:
      continue
    for typeDir in os.listdir():
      os.chdir(typeDir)
      print(os.getcwd(),typeDir,'in t')
      for patientDir in os.listdir():
        os.chdir(patientDir)
        print(os.getcwd(),typeDir,os.listdir(),'in p')
        for x in os.listdir():
          if x == '400X':
            os.chdir(x)
            print('******', os.getcwd() , '******')
            for f in os.listdir():
              file = os.getcwd() + '/' + f
              print(file)
              im = cv2.imread(file)
              pyplot.imsave(dest + f, im)
            os.chdir(os.pardir)
            break
          else:
            continue
          os.chdir(os.pardir)
          break
        os.chdir(os.pardir)
        break
      os.chdir(os.pardir)
      break
    os.chdir(os.pardir)
  os.chdir(os.pardir)
os.getcwd()

/content/drive/MyDrive/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma ductal_carcinoma in t
/content/drive/MyDrive/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-11951 ductal_carcinoma ['400X', '40X', '100X', '200X'] in p
****** /content/drive/MyDrive/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-11951/400X ******
/content/drive/MyDrive/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-11951/400X/SOB_M_DC-14-11951-400-001.png
/content/drive/MyDrive/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-11951/400X/SOB_M_DC-14-11951-400-004.png
/content/drive/MyDrive/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-11951/400X/SOB_M_DC-14-11951-400-002.png
/content/drive/MyDrive/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-11951/400X/SOB_M_DC-14-11951-400-003.png
/content/drive/MyDrive/Brea

'/content/drive/MyDrive/BreaKHis_v1/histology_slides/breast'